# Reward Recommendation Algorithm Prototype

This notebook implements and evaluates different algorithms for reward recommendation.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import json
import os
from datetime import datetime

# For collaborative filtering
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## Load and Prepare Data

In [ ]:
# Load customers, rewards and events data
with open('../data/customers.json', 'r') as f:
    customers = json.load(f)
    
with open('../data/rewards.json', 'r') as f:
    rewards = json.load(f)
    
with open('../data/events.json', 'r') as f:
    events = json.load(f)
    
# Convert to DataFrames
customers_df = pd.DataFrame(customers)
rewards_df = pd.DataFrame(rewards)
events_df = pd.DataFrame(events)

print(f"Loaded {len(customers_df)} customers, {len(rewards_df)} rewards, and {len(events_df)} events")

In [ ]:
# Filter events to get reward claims
claims_df = events_df[events_df['event_type'] == 'reward_claim'].copy()

# Extract reward ID from metadata
claims_df['reward_id'] = claims_df['metadata'].apply(lambda x: x.get('reward_id', None))

# Drop rows with missing reward_id
claims_df = claims_df.dropna(subset=['reward_id'])

# Create interactions dataframe for collaborative filtering
interactions_df = claims_df[['customer_id', 'reward_id']].copy()

# Add implicit rating (1 for claimed)
interactions_df['rating'] = 1.0

print(f"Created {len(interactions_df)} customer-reward interactions")
interactions_df.head()

In [ ]:
# Extract customer features for content-based filtering
def extract_customer_features(customer):
    features = {}
    
    # Extract demographic features
    attributes = customer.get('attributes', {})
    features['age'] = attributes.get('age', 35)  # Default age
    
    # One-hot encode gender
    gender = attributes.get('gender', 'unknown')
    features['gender_male'] = 1 if gender == 'male' else 0
    features['gender_female'] = 1 if gender == 'female' else 0
    features['gender_other'] = 1 if gender not in ['male', 'female', 'unknown'] else 0
    
    # One-hot encode location
    location = attributes.get('location', 'unknown')
    features['location'] = location
    
    # Process interests
    interests = attributes.get('interests', [])
    features['interest_count'] = len(interests)
    
    # Track specific interests
    common_interests = ['fashion', 'technology', 'sports', 'beauty', 'home', 'travel', 'food']
    for interest in common_interests:
        features[f'interest_{interest}'] = 1 if interest in interests else 0
    
    return features

# Extract reward features for content-based filtering
def extract_reward_features(reward):
    features = {}
    
    # Basic attributes
    features['value'] = reward.get('value', 0)
    
    # One-hot encode type
    reward_type = reward.get('type', 'unknown')
    features['type'] = reward_type
    
    # Extract conditions
    conditions = reward.get('conditions', {})
    features['has_min_purchase'] = 1 if 'min_purchase' in conditions else 0
    features['min_purchase_value'] = conditions.get('min_purchase', 0) if 'min_purchase' in conditions else 0
    
    return features

# Extract customer features
customer_features = {}
for customer in customers:
    customer_id = customer.get('id')
    customer_features[customer_id] = extract_customer_features(customer)

# Extract reward features
reward_features = {}
for reward in rewards:
    reward_id = reward.get('id')
    reward_features[reward_id] = extract_reward_features(reward)

# Convert to DataFrames
customer_features_df = pd.DataFrame.from_dict(customer_features, orient='index')
reward_features_df = pd.DataFrame.from_dict(reward_features, orient='index')

print(f"Extracted features for {len(customer_features_df)} customers and {len(reward_features_df)} rewards")
customer_features_df.head()

## Implement Recommendation Algorithms

In [ ]:
# 1. Popularity-based recommendation
def popularity_recommender(interactions_df, top_n=5):
    """Recommend the most popular rewards."""
    # Count claims per reward
    reward_popularity = interactions_df['reward_id'].value_counts().reset_index()
    reward_popularity.columns = ['reward_id', 'claim_count']
    
    # Sort by popularity
    reward_popularity = reward_popularity.sort_values('claim_count', ascending=False)
    
    # Return top N rewards
    return reward_popularity.head(top_n)['reward_id'].tolist()

# 2. Collaborative Filtering with Surprise
def collaborative_filtering_recommender(interactions_df, customer_id, reward_ids, top_n=5):
    """Recommend rewards using collaborative filtering."""
    # Prepare data for Surprise
    reader = Reader(rating_scale=(0, 1))
    data = Dataset.load_from_df(interactions_df[['customer_id', 'reward_id', 'rating']], reader)
    
    # Train model
    algo = SVD()
    trainset = data.build_full_trainset()
    algo.fit(trainset)
    
    # Generate predictions for all rewards not yet claimed by the customer
    claimed_rewards = interactions_df[interactions_df['customer_id'] == customer_id]['reward_id'].tolist()
    recommendations = []
    
    for reward_id in reward_ids:
        if reward_id not in claimed_rewards:
            # Predict rating
            prediction = algo.predict(customer_id, reward_id)
            recommendations.append((reward_id, prediction.est))
    
    # Sort by predicted rating
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    # Return top N rewards
    return [rec[0] for rec in recommendations[:top_n]]

# 3. Content-based recommendation
def content_based_recommender(customer_id, customer_features_df, reward_features_df, top_n=5):
    """Recommend rewards based on customer-reward feature similarity."""
    # Get customer features
    if customer_id not in customer_features_df.index:
        return []  # Customer not found
        
    customer_vec = customer_features_df.loc[customer_id]
    
    # Calculate similarity scores for each reward
    similarity_scores = []
    
    for reward_id, reward_vec in reward_features_df.iterrows():
        score = 0.0
        
        # Base score on reward value (higher is better, but diminishing returns)
        score += np.log1p(reward_vec['value']) * 0.3
        
        # Adjust for minimum purchase requirement
        if reward_vec['has_min_purchase'] == 1:
            score -= np.log1p(reward_vec['min_purchase_value']) * 0.1
        
        # Boost score for rewards matching customer interests
        interest_columns = [col for col in customer_vec.index if col.startswith('interest_') and col != 'interest_count']
        interests_match = False
        
        for col in interest_columns:
            if customer_vec[col] == 1:
                interest = col.replace('interest_', '')
                if interest in reward_vec['type'].lower():
                    interests_match = True
                    break
                    
        if interests_match:
            score += 1.0
            
        # Gender-specific adjustments (example)
        if customer_vec['gender_female'] == 1 and 'beauty' in reward_vec['type'].lower():
            score += 0.5
            
        if customer_vec['gender_male'] == 1 and 'sports' in reward_vec['type'].lower():
            score += 0.5
            
        # Age-based adjustments
        age = customer_vec['age']
        if age < 30 and 'technology' in reward_vec['type'].lower():
            score += 0.5
        elif age > 50 and 'home' in reward_vec['type'].lower():
            score += 0.5
            
        similarity_scores.append((reward_id, score))
    
    # Sort by similarity score
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    
    # Return top N rewards
    return [rec[0] for rec in similarity_scores[:top_n]]

# 4. Hybrid recommender (combining collaborative and content-based)
def hybrid_recommender(customer_id, interactions_df, customer_features_df, reward_features_df, top_n=5):
    """Hybrid recommendation combining collaborative and content-based approaches."""
    # Get recommendations from both methods
    try:
        collab_recs = collaborative_filtering_recommender(
            interactions_df, customer_id, reward_features_df.index.tolist(), top_n=top_n*2
        )
    except:
        # Fallback if collaborative filtering fails
        collab_recs = []
        
    content_recs = content_based_recommender(
        customer_id, customer_features_df, reward_features_df, top_n=top_n*2
    )
    
    # Combine recommendations with weighting
    combined_scores = {}
    
    # Score collaborative recommendations
    for i, reward_id in enumerate(collab_recs):
        # Reverse rank scoring (higher rank = higher score)
        combined_scores[reward_id] = combined_scores.get(reward_id, 0) + (top_n*2 - i) * 0.6
        
    # Score content-based recommendations
    for i, reward_id in enumerate(content_recs):
        # Reverse rank scoring (higher rank = higher score)
        combined_scores[reward_id] = combined_scores.get(reward_id, 0) + (top_n*2 - i) * 0.4
        
    # Sort by combined score
    sorted_rewards = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Return top N rewards
    return [rec[0] for rec in sorted_rewards[:top_n]]

# 5. Context-aware recommender with multi-armed bandit approach
class ContextualBanditRecommender:
    """Context-aware recommender using Thompson Sampling."""
    
    def __init__(self, reward_ids):
        self.reward_ids = reward_ids
        self.alpha = {reward_id: 1.0 for reward_id in reward_ids}  # Successes
        self.beta = {reward_id: 1.0 for reward_id in reward_ids}   # Failures
        self.context_weights = {}  # Contextual weights
        
    def update(self, reward_id, success, context=None):
        """Update model based on reward outcome."""
        if success:
            self.alpha[reward_id] += 1.0
        else:
            self.beta[reward_id] += 1.0
            
        # Update contextual weights if context provided
        if context is not None:
            for key, value in context.items():
                context_key = f"{key}:{value}"
                if context_key not in self.context_weights:
                    self.context_weights[context_key] = {reward_id: {'alpha': 1.0, 'beta': 1.0} for reward_id in self.reward_ids}
                    
                if success:
                    self.context_weights[context_key][reward_id]['alpha'] += 1.0
                else:
                    self.context_weights[context_key][reward_id]['beta'] += 1.0
    
    def recommend(self, top_n=5, context=None):
        """Recommend rewards using Thompson Sampling."""
        # Sample from beta distributions
        samples = {}
        
        for reward_id in self.reward_ids:
            # Base sample
            base_sample = np.random.beta(self.alpha[reward_id], self.beta[reward_id])
            samples[reward_id] = base_sample
            
            # Apply contextual adjustments if context provided
            if context is not None:
                context_adjustment = 0.0
                context_count = 0
                
                for key, value in context.items():
                    context_key = f"{key}:{value}"
                    if context_key in self.context_weights and reward_id in self.context_weights[context_key]:
                        context_weights = self.context_weights[context_key][reward_id]
                        context_sample = np.random.beta(context_weights['alpha'], context_weights['beta'])
                        context_adjustment += context_sample
                        context_count += 1
                        
                if context_count > 0:
                    context_adjustment /= context_count
                    samples[reward_id] = 0.7 * base_sample + 0.3 * context_adjustment
        
        # Sort by sampled values
        sorted_samples = sorted(samples.items(), key=lambda x: x[1], reverse=True)
        
        # Return top N rewards
        return [rec[0] for rec in sorted_samples[:top_n]]

# Initialize bandit recommender
bandit_recommender = ContextualBanditRecommender(reward_features_df.index.tolist())

# Simulate some learning for the bandit
for _ in range(100):
    # Randomly select a customer and context
    customer_id = np.random.choice(customer_features_df.index)
    customer_data = customer_features_df.loc[customer_id]
    
    context = {
        'age_group': 'young' if customer_data['age'] < 30 else 'middle' if customer_data['age'] < 60 else 'senior',
        'gender': 'male' if customer_data['gender_male'] == 1 else 'female' if customer_data['gender_female'] == 1 else 'other'
    }
    
    # Recommend a reward
    recommendations = bandit_recommender.recommend(top_n=1, context=context)
    if recommendations:
        reward_id = recommendations[0]
        
        # Simulate outcome (more likely to succeed if matching interests or high value)
        reward_data = reward_features_df.loc[reward_id]
        
        # Base success probability
        success_prob = 0.3
        
        # Adjust for reward value
        success_prob += min(0.3, reward_data['value'] / 100.0)
        
        # Adjust for demographic match
        if (context['gender'] == 'female' and 'beauty' in reward_data['type'].lower()) or \
           (context['gender'] == 'male' and 'sports' in reward_data['type'].lower()):
            success_prob += 0.2
            
        # Simulate outcome
        success = np.random.random() < success_prob
        
        # Update model
        bandit_recommender.update(reward_id, success, context)

## Evaluate Recommendation Algorithms

In [ ]:
# Create evaluation dataset
# Split interactions into train and test
train_df, test_df = train_test_split(interactions_df, test_size=0.2, random_state=42)

print(f"Training set: {len(train_df)} interactions")
print(f"Test set: {len(test_df)} interactions")

In [ ]:
# Evaluate popularity-based recommender
popular_rewards = popularity_recommender(train_df, top_n=10)
print(f"Top 10 popular rewards: {popular_rewards}")

# Calculate hit rate (how often the claimed reward is in the recommendations)
hit_count = 0
for _, row in test_df.iterrows():
    if row['reward_id'] in popular_rewards:
        hit_count += 1
        
popularity_hit_rate = hit_count / len(test_df) if len(test_df) > 0 else 0
print(f"Popularity-based hit rate: {popularity_hit_rate:.4f}")

In [ ]:
# Evaluate collaborative filtering recommender
# Use a subset of test customers for faster evaluation
test_customers = test_df['customer_id'].unique()[:10]
collab_hit_rates = []

for customer_id in test_customers:
    # Get claimed rewards for this customer in test set
    claimed_rewards = test_df[test_df['customer_id'] == customer_id]['reward_id'].tolist()
    
    if not claimed_rewards:
        continue
        
    # Get recommendations
    try:
        recommendations = collaborative_filtering_recommender(
            train_df, customer_id, reward_features_df.index.tolist(), top_n=10
        )
    except:
        continue
        
    # Calculate hit rate
    hits = sum(1 for reward_id in claimed_rewards if reward_id in recommendations)
    hit_rate = hits / len(claimed_rewards) if claimed_rewards else 0
    collab_hit_rates.append(hit_rate)

collab_avg_hit_rate = np.mean(collab_hit_rates) if collab_hit_rates else 0
print(f"Collaborative filtering average hit rate: {collab_avg_hit_rate:.4f}")

In [ ]:
# Evaluate content-based recommender
content_hit_rates = []

for customer_id in test_customers:
    # Get claimed rewards for this customer in test set
    claimed_rewards = test_df[test_df['customer_id'] == customer_id]['reward_id'].tolist()
    
    if not claimed_rewards:
        continue
        
    # Get recommendations
    recommendations = content_based_recommender(
        customer_id, customer_features_df, reward_features_df, top_n=10
    )
        
    # Calculate hit rate
    hits = sum(1 for reward_id in claimed_rewards if reward_id in recommendations)
    hit_rate = hits / len(claimed_rewards) if claimed_rewards else 0
    content_hit_rates.append(hit_rate)

content_avg_hit_rate = np.mean(content_hit_rates) if content_hit_rates else 0
print(f"Content-based average hit rate: {content_avg_hit_rate:.4f}")

In [ ]:
# Evaluate hybrid recommender
hybrid_hit_rates = []

for customer_id in test_customers:
    # Get claimed rewards for this customer in test set
    claimed_rewards = test_df[test_df['customer_id'] == customer_id]['reward_id'].tolist()
    
    if not claimed_rewards:
        continue
        
    # Get recommendations
    try:
        recommendations = hybrid_recommender(
            customer_id, train_df, customer_features_df, reward_features_df, top_n=10
        )
    except:
        continue
        
    # Calculate hit rate
    hits = sum(1 for reward_id in claimed_rewards if reward_id in recommendations)
    hit_rate = hits / len(claimed_rewards) if claimed_rewards else 0
    hybrid_hit_rates.append(hit_rate)

hybrid_avg_hit_rate = np.mean(hybrid_hit_rates) if hybrid_hit_rates else 0
print(f"Hybrid recommender average hit rate: {hybrid_avg_hit_rate:.4f}")

In [ ]:
# Evaluate contextual bandit recommender
bandit_hit_rates = []

for customer_id in test_customers:
    # Get claimed rewards for this customer in test set
    claimed_rewards = test_df[test_df['customer_id'] == customer_id]['reward_id'].tolist()
    
    if not claimed_rewards or customer_id not in customer_features_df.index:
        continue
        
    # Get customer context
    customer_data = customer_features_df.loc[customer_id]
    
    context = {
        'age_group': 'young' if customer_data['age'] < 30 else 'middle' if customer_data['age'] < 60 else 'senior',
        'gender': 'male' if customer_data['gender_male'] == 1 else 'female' if customer_data['gender_female'] == 1 else 'other'
    }
    
    # Get recommendations
    recommendations = bandit_recommender.recommend(top_n=10, context=context)
        
    # Calculate hit rate
    hits = sum(1 for reward_id in claimed_rewards if reward_id in recommendations)
    hit_rate = hits / len(claimed_rewards) if claimed_rewards else 0
    bandit_hit_rates.append(hit_rate)

bandit_avg_hit_rate = np.mean(bandit_hit_rates) if bandit_hit_rates else 0
print(f"Contextual bandit recommender average hit rate: {bandit_avg_hit_rate:.4f}")

## Compare Algorithms

In [ ]:
# Compare hit rates
algorithms = ['Popularity', 'Collaborative', 'Content-based', 'Hybrid', 'Contextual Bandit']
hit_rates = [popularity_hit_rate, collab_avg_hit_rate, content_avg_hit_rate, hybrid_avg_hit_rate, bandit_avg_hit_rate]

plt.figure(figsize=(10, 6))
plt.bar(algorithms, hit_rates, color='skyblue')
plt.title('Recommendation Algorithm Performance Comparison')
plt.xlabel('Algorithm')
plt.ylabel('Hit Rate')
plt.ylim(0, max(hit_rates) * 1.2)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add values on top of bars
for i, v in enumerate(hit_rates):
    plt.text(i, v + 0.01, f"{v:.4f}", ha='center')
    
plt.tight_layout()
plt.show()

## Implement Final Hybrid Recommender

In [ ]:
def final_recommender(customer_id, interactions_df, customer_features_df, reward_features_df, reward_bandit=None, top_n=5):
    """Final hybrid recommendation algorithm combining all approaches."""
    # Try collaborative filtering
    try:
        collab_recs = collaborative_filtering_recommender(
            interactions_df, customer_id, reward_features_df.index.tolist(), top_n=top_n*2
        )
    except:
        # Fallback to popular rewards if collaborative filtering fails
        collab_recs = popularity_recommender(interactions_df, top_n=top_n*2)
        
    # Get content-based recommendations
    if customer_id in customer_features_df.index:
        content_recs = content_based_recommender(
            customer_id, customer_features_df, reward_features_df, top_n=top_n*2
        )
    else:
        content_recs = []
        
    # Get bandit recommendations if available
    if reward_bandit is not None and customer_id in customer_features_df.index:
        customer_data = customer_features_df.loc[customer_id]
        context = {
            'age_group': 'young' if customer_data['age'] < 30 else 'middle' if customer_data['age'] < 60 else 'senior',
            'gender': 'male' if customer_data['gender_male'] == 1 else 'female' if customer_data['gender_female'] == 1 else 'other'
        }
        bandit_recs = reward_bandit.recommend(top_n=top_n*2, context=context)
    else:
        bandit_recs = []
    
    # Combine all recommendations with weights
    combined_scores = {}
    
    # Assign weights to each approach
    weights = {
        'collaborative': 0.4,
        'content': 0.3,
        'bandit': 0.3
    }
    
    # Score collaborative recommendations
    for i, reward_id in enumerate(collab_recs):
        # Reverse rank scoring (higher rank = higher score)
        combined_scores[reward_id] = combined_scores.get(reward_id, 0) + \
                                      (top_n*2 - i) / (top_n*2) * weights['collaborative']
        
    # Score content-based recommendations
    for i, reward_id in enumerate(content_recs):
        # Reverse rank scoring (higher rank = higher score)
        combined_scores[reward_id] = combined_scores.get(reward_id, 0) + \
                                      (top_n*2 - i) / (top_n*2) * weights['content']
        
    # Score bandit recommendations
    for i, reward_id in enumerate(bandit_recs):
        # Reverse rank scoring (higher rank = higher score)
        combined_scores[reward_id] = combined_scores.get(reward_id, 0) + \
                                      (top_n*2 - i) / (top_n*2) * weights['bandit']
        
    # Sort by combined score
    sorted_rewards = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Return top N rewards
    return [rec[0] for rec in sorted_rewards[:top_n]]

# Test final recommender
test_customer = test_customers[0] if test_customers else customer_features_df.index[0]
final_recommendations = final_recommender(
    test_customer, interactions_df, customer_features_df, reward_features_df, 
    reward_bandit=bandit_recommender, top_n=5
)

print(f"Final recommendations for customer {test_customer}:")
for i, reward_id in enumerate(final_recommendations):
    reward_name = rewards_df[rewards_df['id'] == reward_id]['name'].values[0] if reward_id in rewards_df['id'].values else "Unknown"
    print(f"{i+1}. {reward_name} (ID: {reward_id})")

## Export Final Recommender Model

In [ ]:
import pickle

# Create a model package for export
recommender_package = {
    'bandit_recommender': bandit_recommender,
    'customer_features_df': customer_features_df,
    'reward_features_df': reward_features_df,
    'interactions_df': interactions_df,
    'recommender_function': final_recommender
}

# Save the model package
with open('../data/processed/reward_recommender_model.pkl', 'wb') as f:
    pickle.dump(recommender_package, f)
    
print("Saved recommender model to '../data/processed/reward_recommender_model.pkl'")